# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do


# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.


In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n**0.5
        n += 1


# Example usage:
limit = 5
generator = square_root_generator(limit)

total = 0
for sqrt_value in generator:
    # print(sqrt_value)
    total += sqrt_value
print(total)

8.382332347441762


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n**0.5
        n += 1


# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


1. 8.382332347441762
2. 3.605551275463989


# 2. Append a generator to a table with existing data

Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**


# Load first data

In [28]:
import dlt


def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


people1_pipeline = dlt.pipeline(
    pipeline_name="people_etl", dataset_name="people", destination="duckdb"
)
info = people1_pipeline.run(
    people_1(), table_name="people_table", write_disposition="replace"
)

In [36]:
import duckdb

conn = duckdb.connect(f"{people1_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path= {people1_pipeline.dataset_name}")
display(conn.sql('SELECT * FROM "people_table"'))
sum_age1 = conn.sql('SELECT sum(age) FROM "people_table"')
print(f"sum of the age in first table entry:{sum_age1}")

┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707694671.879852 │ ys4FmjPwVcKmxg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707694671.879852 │ gX1BQ0zEE+uKcQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707694671.879852 │ ta7ovZpW68+quQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707694671.879852 │ zeRzJYcflLXXdQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707694671.879852 │ 1BNquTLNycI1Og │ NULL       │
└───────┴──────────┴───────┴─────────┴───────────────────┴────────────────┴────────────┘

sum of the age in first table entry:┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘



# Load second data


In [30]:
def people_2():
    for i in range(3, 9):
        yield {
            "ID": i,
            "Name": f"Person_{i}",
            "Age": 30 + i,
            "City": "City_B",
            "Occupation": f"Job_{i}",
        }


people2_pipeline = people1_pipeline.run(
    people_2(), table_name="people_table", write_disposition="append"
)

In [32]:
conn = duckdb.connect(f"{people1_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path= {people2_pipeline.dataset_name}")
display(conn.sql('SELECT * FROM "people_table"'))
sum_age1 = conn.sql('SELECT sum(age) FROM "people_table"')
print(f"sum of the age in second table entry:{sum_age1}")

┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707692518.112243 │ xV/dvgCXLK54UQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707692518.112243 │ 5/WwlQ+sgamfHQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707692518.112243 │ +/e5xHRaDsQdUA │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707692518.112243 │ rOOYnjYyQnMQWQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707692518.112243 │ U2iZ6DRIG3qCZg │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707692699.131199 │ bFn51MIJj5a+Tg │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707692699.131199 │ c0246L4tOSLJXw │ Job_4      │
│     5 │ Person_5 │ 

sum of the age in second table entry:┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



answer= 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:


In [4]:
%%capture
!pip install dlt[duckdb]
import dlt
import duckdb

for first running set write dispo to replace to replace fresh data
When loading first data we can using primary key or just set using @dlt.resources to merge the second data

The difference between these approaches lies primarily in the criteria used to identify existing records. Using the primary key ensures uniqueness based on the primary key constraint of the table, while using a merge key provides flexibility to define custom criteria for identifying existing records, which may not necessarily be unique.

In summary, while both options aim to merge new data with existing data in the target table, they differ in the criteria used to identify existing records for updates.

In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


people1_pipeline = dlt.pipeline(
    pipeline_name="people_etl", dataset_name="people", destination="duckdb"
)
info = people1_pipeline.run(
    people_1(),
    table_name="people_table",
    write_disposition="replace",
    primary_key=["ID"],
)

In [6]:
# @dlt.resource(write_disposition="merge", merge_key=["ID"])
def people_2():
    for i in range(3, 9):
        yield {
            "ID": i,
            "Name": f"Person_{i}",
            "Age": 30 + i,
            "City": "City_B",
            "Occupation": f"Job_{i}",
        }


people2_pipeline = people1_pipeline.run(
    people_2(), table_name="people_table", write_disposition="merge"
)

In [7]:
conn = duckdb.connect(f"{people1_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path= {people2_pipeline.dataset_name}")
display(conn.sql('SELECT * FROM "people_table"'))
sum_age1 = conn.sql('SELECT sum(age) FROM "people_table"')
print(f"sum of the age in second table entry:{sum_age1}")

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707696649.5891662 │ Zq2eMQIfDicdBg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707696649.5891662 │ HZDe25uKguNDCQ │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1707696659.6374679 │ n/6wiGSZ9Qfb2Q │ Job_8      │
│     5 │ Person_5 │    35 │ City_B  │ 1707696659.6374679 │ Z9FIs/os9DUAyQ │ Job_5      │
│     4 │ Person_4 │    34 │ City_B  │ 1707696659.6374679 │ mAgCbLtZxoamJw │ Job_4      │
│     7 │ Person_7 │    37 │ City_B  │ 1707696659.6374679 │ VfjPM64cdUdr4w │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1707696659.6374679 │ xTIHKLxvvZ1OYw │ Job_3      │
│     6 │ 

sum of the age in second table entry:┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘



answer:266

Questions? difficulties? We are here to help.

- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX
